In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [4]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['pytorch_geometric_test.pkl',
 'cis-trans-isomerism-feature_test.pkl',
 'pytorch_geometric_train.pkl',
 'laplaction_matrix_train.pkl',
 'feature-engineering-physical-chemical-measurement_test.pkl',
 'laplaction_matrix_test.pkl',
 'cis-trans-isomerism-feature_train.pkl',
 'feature-engineering-physical-chemical-measurement_train.pkl',
 'stanislav-blinov_train.pkl',
 'stanislav-blinov_test.pkl']

In [5]:
file_list = ['pytorch_geometric_test.pkl',
 'cis-trans-isomerism-feature_test.pkl',
 'pytorch_geometric_train.pkl',
 'laplaction_matrix_train.pkl',
 'feature-engineering-physical-chemical-measurement_test.pkl',
 'laplaction_matrix_test.pkl',
 'cis-trans-isomerism-feature_train.pkl',
 'feature-engineering-physical-chemical-measurement_train.pkl',
 'stanislav-blinov_train.pkl',
 'stanislav-blinov_test.pkl']
print(len(file_list))

10


In [6]:
trail = []
def prepare_data(feature_folder='../../data/feature', csv_file_folder='../../data/input', feature_file_list=None, trail=trail):
    
    df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip').rename(columns={'index': 'id'})
    df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip').rename(columns={'index': 'id'})
    
#     if os.path.exists(f'{feature_folder}/df_train'):
#         print(f'=========================load from temp===============================')
#         df_train = pd.read_pickle(f'{feature_folder}/df_train')
#         df_test = pd.read_pickle(f'{feature_folder}/df_test')
#         return df_train, df_test
    
#     df_train = pd.read_csv(f"{csv_file_folder}/train.csv")
#     df_test = pd.read_csv(f"{csv_file_folder}/test.csv")
#     scalar_coupling_contributions = pd.read_csv(f'{csv_file_folder}/scalar_coupling_contributions.csv')
    
#     #scalar_coupling_constant fc sd pso dso
#     df_train = pd.merge(df_train, scalar_coupling_contributions, how = 'left',
#                   left_on  = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'],
#                   right_on = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'])
    
    print(df_train.shape, df_test.shape)

    if type(feature_file_list) == type(None):
        feature_file_list = os.listdir(feature_folder)

    for f in feature_file_list:
        print(f'========================={f}===============================')
        if (f.endswith('.pkl')) and (not f.startswith('.')):
            if f[:-4].endswith('train'):
                df_feature_i = pd.read_pickle(f'{feature_folder}/{f}').sort_values(by=['id'])
                df_feature_i = df_feature_i.reset_index(drop=True)
                columns_i = df_feature_i.columns.tolist()
                new_columns = set(columns_i) - set(df_train.columns.tolist())
                duplicates_columns = [col for col in columns_i if col not in list(new_columns)]
                for col in duplicates_columns:
                    try:
                        error = np.where(df_feature_i[col].values!=df_train[col].values)[0]
                        if error.shape[0] > 0:
                            if np.where(~np.isnan(df_feature_i[col].values[np.where(df_feature_i[col].values!=df_train[col].values)[0]]))[0].shape[0]>0:
                                if not str in [type(df_train[col].values[0]),type(df_feature_i[col].values[0])]:
                                    trail.append({'df_feature_i':df_feature_i, 'df_train':df_train})
                                    print(col, error, [f'{v1}:{v2}' for v1, v2 in zip(df_feature_i[col].values[error], df_train[col].values[error])])
                                    raise Exception()
                    except Exception as e:
#                         raise Exception(col)
                        print(col)
                df_train = pd.merge(df_train, df_feature_i[list(new_columns) + ['id']], on='id')
                df_train = df_train.sort_values(by=['id'])
                df_train = df_train.reset_index(drop=True)
                print('train add', f, new_columns)
            if f[:-4].endswith('test'):
                df_feature_i = pd.read_pickle(f'{feature_folder}/{f}').sort_values(by=['id'])
                df_feature_i = df_feature_i.reset_index(drop=True)
                columns_i = df_feature_i.columns.tolist()
                new_columns = set(columns_i) - set(df_test.columns.tolist())
                duplicates_columns = [col for col in columns_i if col not in list(new_columns)]
                for col in duplicates_columns:
                    try:
                        error = np.where(df_feature_i[col].values!=df_test[col].values)[0]
                        if error.shape[0] > 0:
                            if np.where(~np.isnan(df_feature_i[col].values[np.where(df_feature_i[col].values!=df_test[col].values)[0]]))[0].shape[0]>0:
                                if not str in [type(df_test[col].values[0]),type(df_feature_i[col].values[0])]:
                                    trail.append({'df_feature_i':df_feature_i, 'df_test':df_test})
                                    print(col, error, [f'{v1}:{v2}' for v1, v2 in zip(df_feature_i[col].values[error], df_test[col].values[error])])
                                    raise Exception()
                    except Exception as e:
#                         raise Exception(col)
                        print(col)
                df_test = pd.merge(df_test, df_feature_i[list(new_columns) + ['id']], on='id')
                df_test = df_test.sort_values(by=['id'])
                df_test = df_test.reset_index(drop=True)
                print('test add', f, new_columns)

    print(f'=========================encode label===============================')
    numerics = ['int16', 'int8', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df_train.columns:
        col_type = df_train[col].dtypes
        if not col_type in numerics:
            print(col, df_train[col].unique())
            le = LabelEncoder()
            le.fit(list(df_train[col].values) + list(df_test[col].values))
            df_train[col] = le.transform(list(df_train[col].values))
            df_test[col] = le.transform(list(df_test[col].values))
            print(le.classes_)

    print(f'=========================fill nan inf===============================')
    df_train = df_train.replace([np.inf, -np.inf], np.nan)
    df_train = df_train.fillna(0)
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    df_test = df_test.fillna(0)

    print(f'=========================rename===============================')
    df_train = df_train.rename(columns={'id': 'index'}) #'scalar_coupling_constant': 'y'
    df_test = df_test.rename(columns={'id': 'index'})
#     df_train = df_train.rename(columns={'molecule_name':'group'})
#     df_test = df_test.rename(columns={'molecule_name':'group'})
#     df_test = df_test.rename(columns={'cycle_size_mean_x':'atom_index_0_cycle_size_mean', 
#                             'cycle_size_mean_y':'atom_index_1_cycle_size_mean',
#                            'n_cycle_x':'atom_index_0_n_cycle',
#                            'n_cycle_y':'atom_index_1_n_cycle'})

    df_train = df_train.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)
    
    if not os.path.exists(f'{feature_folder}/df_train'):
        print(f'=========================save tmp===============================')
        df_train.to_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
        df_test.to_pickle(f'{file_folder}/df_test2.gzde', compression='gzip')
    return df_train, df_test

In [ ]:
df_train, df_test = prepare_data(feature_file_list=file_list)

(4658147, 417) (2505542, 412)
=========================pytorch_geometric_test.pkl===============================
test add pytorch_geometric_test.pkl {'edge_3', 'edge_0', 'atom_index_0_node_pca', 'atom_index_1_node_pca', 'edge_5', 'edge_2', 'edge_4', 'edge_1'}
=========================cis-trans-isomerism-feature_test.pkl===============================
test add cis-trans-isomerism-feature_test.pkl {'is_cis', 'dihedral', 'is_trans'}
=========================pytorch_geometric_train.pkl===============================
train add pytorch_geometric_train.pkl {'edge_3', 'edge_0', 'atom_index_0_node_pca', 'atom_index_1_node_pca', 'edge_5', 'edge_2', 'edge_4', 'edge_1'}
=========================laplaction_matrix_train.pkl===============================
train add laplaction_matrix_train.pkl {'atom_index_1_p_lap_factors', 'atom_index_0_t_dist_core', 'atom_index_0_p_lap_factors', 'atom_index_0_p_lap_core', 'atom_index_0_p_dist_factors', 'atom_index_1_t_dist_core', 'atom_index_1_t_lap_factors_0', 'ato

In [8]:
123

123

In [9]:
df_train.shape

(4658147, 536)

In [10]:
df_test.shape

(2505542, 531)